Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.3 MB/s eta 0:00:00


Cirq — 3-qubit QFT, adım adım ara durumlar

In [2]:
# requirements: pip install cirq numpy
import cirq
import numpy as np

def controlled_R(k, control, target):
    """Controlled R_k where R_k = diag(1, exp(2pi i / 2^k)).
       Cirq CPhase uses angle theta so that |11> gets e^{i theta}.
       We want theta = 2*pi / 2^k.
    """
    theta = 2 * np.pi / (2 ** k)
    return cirq.CPhase(theta).on(control, target)

def qft_stepwise_cirq(x_bits):
    # qubit ordering: q0 = x1 (MSB), q1 = x2, q2 = x3 (LSB)
    q0, q1, q2 = cirq.LineQubit.range(3)
    qubits = [q0, q1, q2]

    # prepare input basis state |x1 x2 x3>
    circuit_prep = cirq.Circuit()
    for qb, bit in zip(qubits, x_bits):
        if bit == 1:
            circuit_prep.append(cirq.X(qb))

    # We'll build circuit incrementally and simulate after each main step
    sim = cirq.Simulator()
    print("Input bits:", x_bits)
    print("Initial statevector (computational basis):")
    print(np.round(sim.simulate(circuit_prep).final_state_vector, 6))

    # Step 1: apply H on q0, then controlled-R2 between q1->q0 and q2->q0
    circ1 = circuit_prep.copy()
    circ1.append(cirq.H(q0))
    circ1.append(controlled_R(2, q1, q0))  # R_2 controlled by q1 on q0
    circ1.append(controlled_R(3, q2, q0))  # R_3 controlled by q2 on q0
    state1 = sim.simulate(circ1).final_state_vector
    print("\nAfter processing q0 (H + controlled R_k from q1,q2):")
    print(np.round(state1, 6))

    # Step 2: apply H on q1, then controlled-R2 from q2->q1
    circ2 = circ1.copy()
    circ2.append(cirq.H(q1))
    circ2.append(controlled_R(2, q2, q1))
    state2 = sim.simulate(circ2).final_state_vector
    print("\nAfter processing q1 (H + controlled R_k from q2):")
    print(np.round(state2, 6))

    # Step 3: apply H on q2 (last qubit)
    circ3 = circ2.copy()
    circ3.append(cirq.H(q2))
    # final swaps for full QFT (swap q0<->q2)
    circ3.append(cirq.SWAP(q0, q2))
    state3 = sim.simulate(circ3).final_state_vector
    print("\nAfter processing q2 (H) and final SWAP (full QFT):")
    print(np.round(state3, 6))

if __name__ == "__main__":
    # örnek girdiler: değiştirerek x1,x2,x3 deneyebilirsin
    qft_stepwise_cirq((0,1,0))  # örn |010>  -> gözlemler q1,q2,q3 dönüşümlerini


Input bits: (0, 1, 0)
Initial statevector (computational basis):
[0.+0.j 1.+0.j]


AttributeError: module 'cirq' has no attribute 'CPhase'

Qiskit — 3-qubit QFT adım adım (ara durumlar)

In [3]:
# requirements: pip install qiskit numpy
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector

def controlled_R_qiskit(qc, k, control, target):
    theta = 2 * np.pi / (2 ** k)   # angle for cp(θ)
    qc.cp(theta, control, target)

def qft_stepwise_qiskit(x_bits):
    # qubit index: 0 -> q0 (MSB x1), 1 -> q1 (x2), 2 -> q2 (LSB x3)
    qc_prep = QuantumCircuit(3)
    for idx, bit in enumerate(x_bits):
        if bit == 1:
            qc_prep.x(idx)

    backend = Aer.get_backend('statevector_simulator')

    print("Input bits:", x_bits)
    sv0 = Statevector.from_instruction(qc_prep)
    print("Initial statevector:", np.round(sv0.data, 6))

    # Step 1: process q0
    qc1 = qc_prep.copy()
    qc1.h(0)
    controlled_R_qiskit(qc1, 2, 1, 0)
    controlled_R_qiskit(qc1, 3, 2, 0)
    sv1 = Statevector.from_instruction(qc1)
    print("\nAfter q0 step (H + controlled R_k from q1,q2):")
    print(np.round(sv1.data, 6))

    # Step 2: process q1
    qc2 = qc1.copy()
    qc2.h(1)
    controlled_R_qiskit(qc2, 2, 2, 1)
    sv2 = Statevector.from_instruction(qc2)
    print("\nAfter q1 step (H + controlled R_k from q2):")
    print(np.round(sv2.data, 6))

    # Step 3: process q2 and swap
    qc3 = qc2.copy()
    qc3.h(2)
    qc3.swap(0, 2)   # final bit reversal
    sv3 = Statevector.from_instruction(qc3)
    print("\nAfter q2 step (H) and swap (full QFT):")
    print(np.round(sv3.data, 6))

if __name__ == "__main__":
    # test örneği: |101>
    qft_stepwise_qiskit((1,0,1))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP — Matris tabanlı, adım adım QFT uygulaması

In [4]:
# requirements: pip install qutip numpy
import numpy as np
from qutip import tensor, basis, qeye, Qobj

# single-qubit gates
H = Qobj([[1,1],[1,-1]]) / np.sqrt(2)
proj0 = Qobj([[1,0],[0,0]])
proj1 = Qobj([[0,0],[0,1]])

def Rk(k):
    """R_k unitary (single qubit)"""
    phase = np.exp(2j * np.pi / (2 ** k))
    return Qobj([[1,0],[0,phase]])

def controlled_Rk_op(n_qubits, control_idx, target_idx, k):
    """Construct controlled-R_k operator on n_qubits via projector decomposition."""
    ops = []
    for i in range(n_qubits):
        if i == control_idx:
            ops.append(proj0)  # when control=0 -> identity on target
        else:
            ops.append(qeye(2))
    P0 = tensor(ops)

    ops = []
    for i in range(n_qubits):
        if i == control_idx:
            ops.append(proj1)  # when control=1 -> apply R_k on target
        elif i == target_idx:
            ops.append(Rk(k))
        else:
            ops.append(qeye(2))
    P1R = tensor(ops)

    return P0 + P1R

def qft_stepwise_qutip(x_bits):
    n = 3
    # build input state |x1 x2 x3>
    states = [basis(2, b) for b in x_bits]
    psi = tensor(*states)
    print("Input bits:", x_bits)
    print("Initial statevector:\n", np.round(psi.full().reshape(-1), 6))

    # Step 1: H on q0 then CR2 (q1->q0) and CR3 (q2->q0)
    U1 = tensor(H, qeye(2), qeye(2))
    CR2 = controlled_Rk_op(3, 1, 0, 2)  # control q1 target q0
    CR3 = controlled_Rk_op(3, 2, 0, 3)  # control q2 target q0
    psi1 = CR3 * (CR2 * (U1 * psi))
    print("\nAfter processing q0 (H + controlled R2,R3):")
    print(np.round(psi1.full().reshape(-1), 6))

    # Step 2: H on q1 then CR2 (q2->q1)
    U2 = tensor(qeye(2), H, qeye(2))
    CR2_q1 = controlled_Rk_op(3, 2, 1, 2)  # control q2 target q1
    psi2 = CR2_q1 * (U2 * psi1)
    print("\nAfter processing q1 (H + controlled R2):")
    print(np.round(psi2.full().reshape(-1), 6))

    # Step 3: H on q2, then swap q0<->q2 for final bit reversal
    U3 = tensor(qeye(2), qeye(2), H)
    psi3 = U3 * psi2
    # swap op (0<->2)
    SWAP_02 = Qobj(np.zeros((8,8), dtype=complex))
    # build SWAP by swapping basis states
    import itertools
    dim = 2**3
    mat = np.zeros((dim,dim), dtype=complex)
    for i in range(dim):
        b = format(i, '03b')
        b_list = list(b)
        # swap positions 0 and 2
        b_list[0], b_list[2] = b_list[2], b_list[0]
        j = int(''.join(b_list), 2)
        mat[j, i] = 1.0
    SWAP_02 = Qobj(mat)
    psi_final = SWAP_02 * psi3
    print("\nAfter processing q2 (H) and final SWAP (full QFT):")
    print(np.round(psi_final.full().reshape(-1), 6))

if __name__ == "__main__":
    # örnek: |011>
    qft_stepwise_qutip((0,1,1))


Input bits: (0, 1, 1)
Initial statevector:
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

After processing q0 (H + controlled R2,R3):
[ 0.      +0.j   0.      +0.j   0.      +0.j   0.707107+0.j
  0.      +0.j   0.      +0.j   0.      +0.j  -0.5     +0.5j]

After processing q1 (H + controlled R2):
[ 0.      +0.j        0.5     +0.j        0.      +0.j
  0.      -0.5j       0.      +0.j       -0.353553+0.353553j
  0.      +0.j        0.353553+0.353553j]


TypeError: incompatible dimensions [[8], [8]] and [[2, 2, 2], [1]]